<a href="https://colab.research.google.com/github/jbischof/keras-nlp/blob/bert_ckpt/bert_base_cased.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install deps

In [1]:
!pip install git+https://github.com/jbischof/keras-nlp.git@bert_ckpt tensorflow tf-models-official --upgrade --quiet

     |████████████████████████████████| 511.7 MB 6.8 kB/s 
     |████████████████████████████████| 2.1 MB 49.8 MB/s 
     |████████████████████████████████| 4.6 MB 45.7 MB/s 
     |████████████████████████████████| 5.8 MB 49.9 MB/s 
     |████████████████████████████████| 1.6 MB 59.9 MB/s 
     |████████████████████████████████| 438 kB 57.4 MB/s 
     |████████████████████████████████| 116 kB 48.3 MB/s 
     |████████████████████████████████| 352 kB 67.8 MB/s 
     |████████████████████████████████| 43 kB 2.1 MB/s 
     |████████████████████████████████| 99 kB 11.3 MB/s 
     |████████████████████████████████| 1.3 MB 51.7 MB/s 
     |████████████████████████████████| 238 kB 73.0 MB/s 
     |████████████████████████████████| 1.1 MB 60.3 MB/s 
     |████████████████████████████████| 636 kB 71.3 MB/s 


In [2]:
import json

import keras_nlp
import tensorflow as tf
import tensorflow_models as tfm
from tensorflow import keras

In [3]:
TOKEN_TYPE = "cased"
MODEL_TYPE = "bert_base"
MODEL_NAME = MODEL_TYPE + "_" + TOKEN_TYPE
VOCAB_SIZE = 28996

## Load the model garden checkpoints and weights

In [4]:
# Model garden BERT paths.
zip_path = f"""https://storage.googleapis.com/tf_model_garden/nlp/bert/v3/{TOKEN_TYPE}_L-12_H-768_A-12.tar.gz"""
zip_file = keras.utils.get_file(
    f"""/content/{MODEL_NAME}""",
    zip_path,
    extract=True,
    archive_format="tar",
)

401886519/401886519 [==============================] - 3s 0us/step


In [5]:
!tar -xvf """{MODEL_NAME}"""

tmp/temp_dir/raw/
tmp/temp_dir/raw/vocab.txt
tmp/temp_dir/raw/bert_model.ckpt.index
tmp/temp_dir/raw/bert_model.ckpt.data-00000-of-00001
tmp/temp_dir/raw/bert_config.json


In [6]:
# Model garden BERT paths.
extract_dir = "/content/tmp/temp_dir/raw/"
vocab_path = extract_dir + "vocab.txt"
checkpoint_path = extract_dir + "bert_model.ckpt"
config_path = extract_dir + "bert_config.json"

In [7]:
vars = tf.train.list_variables(checkpoint_path)
weights = {}
for name, shape in vars:
    print(name, shape)
    weight = tf.train.load_variable(checkpoint_path, name)
    weights[name] = weight

_CHECKPOINTABLE_OBJECT_GRAPH []
encoder/layer_with_weights-0/embeddings/.ATTRIBUTES/VARIABLE_VALUE [28996, 768]
encoder/layer_with_weights-1/embeddings/.ATTRIBUTES/VARIABLE_VALUE [512, 768]
encoder/layer_with_weights-10/_attention_layer/_key_dense/bias/.ATTRIBUTES/VARIABLE_VALUE [12, 64]
encoder/layer_with_weights-10/_attention_layer/_key_dense/kernel/.ATTRIBUTES/VARIABLE_VALUE [768, 12, 64]
encoder/layer_with_weights-10/_attention_layer/_output_dense/bias/.ATTRIBUTES/VARIABLE_VALUE [768]
encoder/layer_with_weights-10/_attention_layer/_output_dense/kernel/.ATTRIBUTES/VARIABLE_VALUE [12, 64, 768]
encoder/layer_with_weights-10/_attention_layer/_query_dense/bias/.ATTRIBUTES/VARIABLE_VALUE [12, 64]
encoder/layer_with_weights-10/_attention_layer/_query_dense/kernel/.ATTRIBUTES/VARIABLE_VALUE [768, 12, 64]
encoder/layer_with_weights-10/_attention_layer/_value_dense/bias/.ATTRIBUTES/VARIABLE_VALUE [12, 64]
encoder/layer_with_weights-10/_attention_layer/_value_dense/kernel/.ATTRIBUTES/VARIABLE

## Load BertBase model with KerasNLP.

In [8]:
model = keras_nlp.models.BertBase(vocabulary_size=VOCAB_SIZE)
model.summary()

Model: "bert"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 token_ids (InputLayer)         [(None, None)]       0           []                               
                                                                                                  
 token_embedding (Embedding)    (None, None, 768)    22268928    ['token_ids[0][0]']              
                                                                                                  
 segment_ids (InputLayer)       [(None, None)]       0           []                               
                                                                                                  
 position_embedding (PositionEm  (None, None, 768)   393216      ['token_embedding[0][0]']        
 bedding)                                                                                      

## Convert Weights

In [9]:
model.get_layer("token_embedding").embeddings.assign(
    weights[
        "encoder/layer_with_weights-0/embeddings/.ATTRIBUTES/VARIABLE_VALUE"
    ]
)
model.get_layer("position_embedding").position_embeddings.assign(
    weights[
        "encoder/layer_with_weights-1/embeddings/.ATTRIBUTES/VARIABLE_VALUE"
    ]
)
model.get_layer("segment_embedding").embeddings.assign(
    weights[
        "encoder/layer_with_weights-2/embeddings/.ATTRIBUTES/VARIABLE_VALUE"
    ]
)
model.get_layer("embeddings_layer_norm").gamma.assign(
    weights["encoder/layer_with_weights-3/gamma/.ATTRIBUTES/VARIABLE_VALUE"]
)
model.get_layer("embeddings_layer_norm").beta.assign(
    weights["encoder/layer_with_weights-3/beta/.ATTRIBUTES/VARIABLE_VALUE"]
)

for i in range(model.num_layers):
    model.get_layer(
        f"transformer_layer_{i}"
    )._self_attention_layer._key_dense.kernel.assign(
        weights[
            f"encoder/layer_with_weights-{i + 4}/_attention_layer/_key_dense/kernel/.ATTRIBUTES/VARIABLE_VALUE"
        ]
    )
    model.get_layer(
        f"transformer_layer_{i}"
    )._self_attention_layer._key_dense.bias.assign(
        weights[
            f"encoder/layer_with_weights-{i + 4}/_attention_layer/_key_dense/bias/.ATTRIBUTES/VARIABLE_VALUE"
        ]
    )
    model.get_layer(
        f"transformer_layer_{i}"
    )._self_attention_layer._query_dense.kernel.assign(
        weights[
            f"encoder/layer_with_weights-{i + 4}/_attention_layer/_query_dense/kernel/.ATTRIBUTES/VARIABLE_VALUE"
        ]
    )
    model.get_layer(
        f"transformer_layer_{i}"
    )._self_attention_layer._query_dense.bias.assign(
        weights[
            f"encoder/layer_with_weights-{i + 4}/_attention_layer/_query_dense/bias/.ATTRIBUTES/VARIABLE_VALUE"
        ]
    )
    model.get_layer(
        f"transformer_layer_{i}"
    )._self_attention_layer._value_dense.kernel.assign(
        weights[
            f"encoder/layer_with_weights-{i + 4}/_attention_layer/_value_dense/kernel/.ATTRIBUTES/VARIABLE_VALUE"
        ]
    )
    model.get_layer(
        f"transformer_layer_{i}"
    )._self_attention_layer._value_dense.bias.assign(
        weights[
            f"encoder/layer_with_weights-{i + 4}/_attention_layer/_value_dense/bias/.ATTRIBUTES/VARIABLE_VALUE"
        ]
    )
    model.get_layer(
        f"transformer_layer_{i}"
    )._self_attention_layer._output_dense.kernel.assign(
        weights[
            f"encoder/layer_with_weights-{i + 4}/_attention_layer/_output_dense/kernel/.ATTRIBUTES/VARIABLE_VALUE"
        ]
    )
    model.get_layer(
        f"transformer_layer_{i}"
    )._self_attention_layer._output_dense.bias.assign(
        weights[
            f"encoder/layer_with_weights-{i + 4}/_attention_layer/_output_dense/bias/.ATTRIBUTES/VARIABLE_VALUE"
        ]
    )
    model.get_layer(
        f"transformer_layer_{i}"
    )._self_attention_layernorm.gamma.assign(
        weights[
            f"encoder/layer_with_weights-{i + 4}/_attention_layer_norm/gamma/.ATTRIBUTES/VARIABLE_VALUE"
        ]
    )
    model.get_layer(
        f"transformer_layer_{i}"
    )._self_attention_layernorm.beta.assign(
        weights[
            f"encoder/layer_with_weights-{i + 4}/_attention_layer_norm/beta/.ATTRIBUTES/VARIABLE_VALUE"
        ]
    )
    model.get_layer(
        f"transformer_layer_{i}"
    )._feedforward_intermediate_dense.kernel.assign(
        weights[
            f"encoder/layer_with_weights-{i + 4}/_intermediate_dense/kernel/.ATTRIBUTES/VARIABLE_VALUE"
        ]
    )
    model.get_layer(
        f"transformer_layer_{i}"
    )._feedforward_intermediate_dense.bias.assign(
        weights[
            f"encoder/layer_with_weights-{i + 4}/_intermediate_dense/bias/.ATTRIBUTES/VARIABLE_VALUE"
        ]
    )
    model.get_layer(
        f"transformer_layer_{i}"
    )._feedforward_output_dense.kernel.assign(
        weights[
            f"encoder/layer_with_weights-{i + 4}/_output_dense/kernel/.ATTRIBUTES/VARIABLE_VALUE"
        ]
    )
    model.get_layer(
        f"transformer_layer_{i}"
    )._feedforward_output_dense.bias.assign(
        weights[
            f"encoder/layer_with_weights-{i + 4}/_output_dense/bias/.ATTRIBUTES/VARIABLE_VALUE"
        ]
    )
    model.get_layer(
        f"transformer_layer_{i}"
    )._feedforward_layernorm.gamma.assign(
        weights[
            f"encoder/layer_with_weights-{i + 4}/_output_layer_norm/gamma/.ATTRIBUTES/VARIABLE_VALUE"
        ]
    )
    model.get_layer(
        f"transformer_layer_{i}"
    )._feedforward_layernorm.beta.assign(
        weights[
            f"encoder/layer_with_weights-{i + 4}/_output_layer_norm/beta/.ATTRIBUTES/VARIABLE_VALUE"
        ]
    )

model.get_layer("pooled_dense").kernel.assign(
    weights["encoder/layer_with_weights-16/kernel/.ATTRIBUTES/VARIABLE_VALUE"]
)
model.get_layer("pooled_dense").bias.assign(
    weights["encoder/layer_with_weights-16/bias/.ATTRIBUTES/VARIABLE_VALUE"]
)
pass

## Compare Output

In [10]:
def preprocess(x):
    tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
        vocabulary=vocab_path,
    )
    packer = keras_nlp.layers.MultiSegmentPacker(
        sequence_length=model.max_sequence_length,
        start_value=tokenizer.token_to_id("[CLS]"),
        end_value=tokenizer.token_to_id("[SEP]"),
    )
    return packer(tokenizer(x))


token_ids, segment_ids = preprocess(["the quick brown fox."])

In [11]:
encoder_config = tfm.nlp.encoders.EncoderConfig(
    type="bert",
    bert=json.load(tf.io.gfile.GFile(config_path)),
)
mg_model = tfm.nlp.encoders.build_encoder(encoder_config)
checkpoint = tf.train.Checkpoint(encoder=mg_model)
checkpoint.read(checkpoint_path).assert_consumed()

In [12]:
keras_nlp_output = model(
    {
        "token_ids": token_ids,
        "segment_ids": segment_ids,
        "padding_mask": token_ids != 0,
    }
)["pooled_output"]

mg_output = mg_model(
    {
        "input_word_ids": token_ids,
        "input_type_ids": segment_ids,
        "padding_mask": token_ids != 0,
    }
)["pooled_output"]

In [27]:
keras_nlp_output[0, 0:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([-0.81249267,  0.54269457,  0.9999706 , -0.9970537 ,  0.97125727,
        0.6073321 ,  0.9917275 , -0.95177907, -0.9899817 , -0.61669564],
      dtype=float32)>

In [28]:
mg_output[0, 0:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([-0.81249064,  0.54269207,  0.9999706 , -0.9970535 ,  0.97125703,
        0.6073294 ,  0.99172735, -0.9517791 , -0.9899815 , -0.6166962 ],
      dtype=float32)>

In [15]:
# Very close! Though not 100% exact.
tf.reduce_mean(keras_nlp_output - mg_output)

<tf.Tensor: shape=(), dtype=float32, numpy=-3.0643907e-08>

In [16]:
# Save BertBase checkpoint
model.save_weights(f"""{MODEL_NAME}.h5""")

In [17]:
model2 = keras_nlp.models.BertBase(vocabulary_size=VOCAB_SIZE)
model2.load_weights(f"""{MODEL_NAME}.h5""")

In [18]:
# Same output from loaded checkpoint
keras_nlp_output2 = model2(
    {
        "token_ids": token_ids,
        "segment_ids": segment_ids,
        "padding_mask": token_ids != 0,
    }
)["pooled_output"]
tf.reduce_mean(keras_nlp_output - keras_nlp_output2)

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

In [19]:
# Save vocab file as well
vocab_info = tf.io.gfile.GFile(vocab_path).read()
f = open("vocab.txt", "w")
f.write(vocab_info)

210441

In [20]:
# Get MD5 of model
!md5sum """{MODEL_NAME}.h5"""

f30ac6fac11115322e3d0c61e87e98b2  bert_base_cased.h5


In [21]:
# Upload model to drive
# from google.colab import drive
# drive.mount('/content/drive')

In [22]:
# Check uploaded model once added to repo
model_cloud = keras_nlp.models.BertBase(weights=MODEL_NAME)

433474808/433474808 [==============================] - 5s 0us/step


In [23]:
# Same output from cloud model
keras_nlp_output_cloud = model_cloud(
    {
        "token_ids": token_ids,
        "segment_ids": segment_ids,
        "padding_mask": token_ids != 0,
    }
)["pooled_output"]
tf.reduce_mean(keras_nlp_output - keras_nlp_output_cloud)

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

In [26]:
keras_nlp_output_cloud[0, 0:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([-0.81249267,  0.54269457,  0.9999706 , -0.9970537 ,  0.97125727,
        0.6073321 ,  0.9917275 , -0.95177907, -0.9899817 , -0.61669564],
      dtype=float32)>